# Overview

This example demonstrates how to create and run DFT jobs.

1. Import required packages. Adjust [settings](../settings.ipynb) as necessary.

In [ ]:
import nbimporter
from tabulate import tabulate
nbimporter.options['only_defs'] = False

from settings import *
from endpoints.jobs import JobEndpoints
from endpoints.utils import flatten_material
from endpoints.materials import MaterialEndpoints
from endpoints.raw_properties import RawPropertiesEndpoints

2. Set Parameters

    - **MATERIALS_PROJECT_API_KEY**: Your materials project API key.
    
    - **MATERIALS_PROJECT_IDS**: A list of material IDs you would like to import.
    
    - **TAGS**: A list of tags you want to assign to imported materials.

In [ ]:
MATERIALS_PROJECT_API_KEY = "NAZfue7aCq"
MATERIALS_PROJECT_IDS = ["mp-978534", "mp-1096549"]
TAGS = ["phase-ii", "difficulty-1"]

PPN = "1"
QUEUE = "D"
NODES = "1"
TIME_LIMIT = "01:00:00"
CLUSTER = "master-vagrant-cluster-001.exabyte.io"

OWNER_ID = ""
JOB_PREFIX = ""
PROJECT_ID = ""
WORKFLOW_ID = ""
JOBS_SET_NAME = "set"
MATERIALS_SET_NAME = "set"

3. Initialize job, material and raw property endpoints.

In [ ]:
job_endpoints = JobEndpoints(HOST, PORT, ACCOUNT_ID, AUTH_TOKEN, VERSION, SECURE)
material_endpoints = MaterialEndpoints(HOST, PORT, ACCOUNT_ID, AUTH_TOKEN, VERSION, SECURE)
raw_property_endpoints = RawPropertiesEndpoints(HOST, PORT, ACCOUNT_ID, AUTH_TOKEN, VERSION, SECURE)

4. Import material from materials project and add the above tags.

In [ ]:
materials = material_endpoints.import_from_materialsproject(MATERIALS_PROJECT_API_KEY, MATERIALS_PROJECT_IDS, OWNER_ID, TAGS)

5. Create materials set and move the materials into the new set.

In [ ]:
materials_set = material_endpoints.create_set({"name": MATERIALS_SET_NAME, "owner": {"_id": OWNER_ID}})
[material_endpoints.move_to_set(m["_id"], "", materials_set["_id"]) for m in materials]

6. Create jobs for the materials above.

In [ ]:
compute = job_endpoints.get_compute(CLUSTER, PPN, NODES, QUEUE, TIME_LIMIT)
jobs = job_endpoints.create_by_ids(materials, WORKFLOW_ID, PROJECT_ID, OWNER_ID, JOB_PREFIX, compute)

7. Create jobs set and move the jobs into it.

In [ ]:
jobs_set = job_endpoints.create_set({"name": JOBS_SET_NAMEt, "projectId": PROJECT_ID, "owner": {"_id": OWNER_ID}})
[job_endpoints.move_to_set(j["_id"], "", jobs_set["_id"]) for j in jobs]

8. Submit the jobs to the cluster.

In [ ]:
[job_endpoints.submit(id) for id in [j["_id"] for j in jobs]]

9. Wait for jobs to finish.

In [ ]:
wait_for_jobs_to_finish(job_endpoints, jobs)

10. Iterate over jobs and extract final structure, pressure, and band gaps.

In [ ]:
rows = []
for job in jobs:

11. Get inital structure.

In [ ]:
    initial_structure = material_endpoints.get(job["_material"]["_id"])

12. Extract final structure.

In [ ]:
    unit_flowchart_id = job["workflow"]["subworkflows"][0]["units"][0]["flowchartId"]
    final_structure = raw_property_endpoints.get_property(job["_id"], unit_flowchart_id, "final_structure")["data"]

13. Extract pressure.

In [ ]:
    unit_flowchart_id = job["workflow"]["subworkflows"][0]["units"][0]["flowchartId"]
    pressure = raw_property_endpoints.get_property(job["_id"], unit_flowchart_id, "pressure")["data"]["value"]

14. Extract band gaps.

In [ ]:
    unit_flowchart_id = job["workflow"]["subworkflows"][1]["units"][1]["flowchartId"]
    band_gap_direct = raw_property_endpoints.get_direct_band_gap(job["_id"], unit_flowchart_id)
    band_gap_indirect = raw_property_endpoints.get_indirect_band_gap(job["_id"], unit_flowchart_id)

15. Flatten the data append it to the rows list.

In [ ]:
    data = flatten_material(initial_structure)
    data.extend(flatten_material(final_structure))
    data.extend([pressure, band_gap_direct, band_gap_indirect])
    rows.append(data)

16. Print the results.

In [ ]:
keys = ["ID", "NAME", "TAGS", "N-SITES", "LAT-A", "LAT-B", "LAT-C", "LAT-ALPHA", "LAT-BETA", "LAT-GAMMA"]
headers = ["-".join(("INI", key)) for key in keys]
headers.extend(["-".join(("FIN", key)) for key in keys])
headers.extend(["PRESSURE", "DIRECT-GAP", "INDIRECT-GAP"])
print tabulate(rows, headers, tablefmt='grid', stralign='center')